In [1]:
import os

In [2]:
os.environ["LANGSMITH_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "PDF-Bot"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")
os.environ["GEMINI_API_KEY"] = os.getenv("GEMINI_API_KEY")

In [3]:
# initiate the llm
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.4)

e:\Langchain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
llm.invoke("What is Generative AI?")

AIMessage(content='**Generative AI** refers to a category of artificial intelligence models that are designed to **create new, original content** rather than simply analyze or classify existing data. Unlike traditional AI that might identify a cat in an image, generative AI can *create* an image of a cat that has never existed before.\n\nHere\'s a breakdown of what that means:\n\n1.  **"Generative" Aspect:**\n    *   It\'s not just retrieving information or performing a fixed task.\n    *   It\'s about producing something novel, whether it\'s text, images, audio, video, code, or other data types.\n    *   The output often mimics the style, characteristics, and patterns of the data it was trained on, but it\'s not a direct copy.\n\n2.  **How it Works (Simplified):**\n    *   **Training:** Generative AI models are trained on vast datasets (e.g., millions of images, billions of text documents). During this training, they learn the underlying patterns, structures, and relationships within 

In [5]:
# initiate the embedding model
from langchain_huggingface import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [6]:
# load the document
from langchain_community.document_loaders import PyPDFLoader

In [7]:
loader = PyPDFLoader("Physics.pdf")
docs = loader.load()

In [8]:
docs

[Document(metadata={'producer': 'PDFium', 'creator': 'PDFium', 'creationdate': 'D:20251216081050', 'source': 'Physics.pdf', 'total_pages': 50, 'page': 0, 'page_label': '1'}, page_content="1\n1.1 Introduction: \nPhysics is a quantitative science, where \nwe measure various physical quantities \nduring experiments. In our day to day life, we \nneed to measure a number of quantities, e.g., \nsize of objects, volume of liquids, amount of \nmatter, weight of vegetables or fruits, body \ntemperature, length of cloth, etc. A measurement \nalways involves a comparison with a standard \nmeasuring unit which is internationally \naccepted. For example, for measuring the mass \nof a given fruit we need standard mass units \nof 1 kg, 500 g, etc. These standards are called \nunits. The measured quantity is expressed in \nterms of a number followed by a corresponding \nunit, e.g., the length of a wire is written as 5 m \nwhere m (metre) is the unit and 5 is the value of \nthe length in that unit. Dif

In [9]:
docs[0].page_content

"1\n1.1 Introduction: \nPhysics is a quantitative science, where \nwe measure various physical quantities \nduring experiments. In our day to day life, we \nneed to measure a number of quantities, e.g., \nsize of objects, volume of liquids, amount of \nmatter, weight of vegetables or fruits, body \ntemperature, length of cloth, etc. A measurement \nalways involves a comparison with a standard \nmeasuring unit which is internationally \naccepted. For example, for measuring the mass \nof a given fruit we need standard mass units \nof 1 kg, 500 g, etc. These standards are called \nunits. The measured quantity is expressed in \nterms of a number followed by a corresponding \nunit, e.g., the length of a wire is written as 5 m \nwhere m (metre) is the unit and 5 is the value of \nthe length in that unit. Different quantities are \nmeasured in different units, e.g. length in metre \n(m), time in seconds (s), mass in kilogram (kg), \netc. The standard measure of any quantity is \ncalled the un

In [10]:
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains import create_retrieval_chain

In [11]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=250)
splits = text_splitter.split_documents(docs)
vector_store = Chroma.from_documents(documents=splits, embedding=embedding)
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k":4})

In [12]:
# create a prompt
system_prompt = (
    "You are an intelligent assistant designed to answer questions based on the content of an educational textbook PDF uploaded by the user. The PDF contains detailed information on various topics, and your goal is to provide accurate, relevant, and clear answers to any questions the user asks, based on the information in the uploaded document."

    "Instructions: 1.Retrieve relevant sections from the uploaded PDF based on the user's question. 2.Analyze the retrieved content to extract the necessary information. 3.Generate a concise, well-structured answer using only the information provided in the uploaded PDF. 4.If the question is unclear or cannot be answered based on the content, politely inform the user that the information is not available in the document."

    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

In [13]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)

In [14]:
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [15]:
response = rag_chain.invoke({"input":"What are system of units?"})
response

{'input': 'What are system of units?',
 'context': [Document(id='37883aa7-2799-4323-a19d-f65fe925db6b', metadata={'total_pages': 50, 'page': 0, 'source': 'Physics.pdf', 'creationdate': 'D:20251216081050', 'creator': 'PDFium', 'producer': 'PDFium', 'page_label': '1'}, page_content='1\n1.1 Introduction: \nPhysics is a quantitative science, where \nwe measure various physical quantities \nduring experiments. In our day to day life, we \nneed to measure a number of quantities, e.g., \nsize of objects, volume of liquids, amount of \nmatter, weight of vegetables or fruits, body \ntemperature, length of cloth, etc. A measurement \nalways involves a comparison with a standard \nmeasuring unit which is internationally \naccepted. For example, for measuring the mass \nof a given fruit we need standard mass units \nof 1 kg, 500 g, etc. These standards are called \nunits. The measured quantity is expressed in \nterms of a number followed by a corresponding \nunit, e.g., the length of a wire is wri

In [16]:
response['answer']

'Systems of units are various standardized collections of units used for measurement. The document mentions the following systems of units:\n\n*   **CGS:** Centimetre Gram Second system\n*   **MKS:** Metre Kilogram Second system\n*   **FPS:** Foot Pound Second system\n*   **SI:** System International\n\nThe CGS, MKS, and FPS systems were used extensively until recently. In 1971, the "International system" of units, known as SI units, was recommended. The SI units use a decimal system, which makes conversions within the system very simple and convenient.'

In [17]:
# adding chat history
 
from langchain_classic.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

In [18]:
contextualize_q_system_prompt = (
    "You are a question rewriter for a retrieval-augmented generation (RAG) system. "
    "The user has uploaded an educational PDF (e.g., a textbook), and all answers "
    "must be based solely on the content of that document.\n\n"
    "Given the chat history and the latest user question—which may contain "
    "references to earlier questions, answers, or implicit context—rewrite the "
    "latest question into a clear, standalone question that can be understood "
    "without the chat history.\n\n"
    "Requirements:\n"
    "- Preserve the original intent and meaning of the user's question.\n"
    "- Resolve pronouns and vague references (e.g., 'this', 'that', 'it', 'the above topic') "
    "using information from the chat history.\n"
    "- Do NOT add new information or assumptions.\n"
    "- Do NOT answer the question.\n"
    "- If the question is already standalone and clear, return it unchanged.\n\n"
    "Output only the rewritten standalone question."
)

In [19]:
# this is ONLY USED FOR HISTORY_AWARE_RETRIEVAL

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ]
)

In [20]:
history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualize_q_prompt)

In [21]:
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x0000028C993678C0>, search_kwargs={'k': 4}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AI

In [22]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"), #same as the previous prompt, but added history to this.
        ("human", "{input}")
    ]
)

In [23]:
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

In [24]:
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [25]:
from langchain_core.messages import AIMessage, HumanMessage

In [26]:
chat_history = []

In [27]:
question1 = "What are errors in measurements?"

In [28]:
ans1 = rag_chain.invoke({"input":question1, "chat_history":chat_history})

In [29]:
chat_history.extend([
        HumanMessage(content=question1),
        AIMessage(content=ans1['answer'])
    ])

In [30]:
chat_history

[HumanMessage(content='What are errors in measurements?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Faulty measurements of physical quantity can lead to errors. These errors are broadly divided into two categories: systematic errors and random errors.', additional_kwargs={}, response_metadata={})]

In [31]:
question2 = "Differentiate between them."

In [32]:
ans2 = rag_chain.invoke({"input":question2, "chat_history":chat_history})

In [33]:
chat_history.extend([
        HumanMessage(content=question2),
        AIMessage(content=ans2['answer'])
    ])

In [34]:
chat_history

[HumanMessage(content='What are errors in measurements?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Faulty measurements of physical quantity can lead to errors. These errors are broadly divided into two categories: systematic errors and random errors.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Differentiate between them.', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Errors in measurements are broadly divided into two categories: Systematic errors and Random errors.\n\nHere's how they differ:\n\n**Systematic Errors:**\n*   **Nature:** These errors are not determined by chance but are introduced by an inaccuracy inherent to the system, either in the observation or measurement process.\n*   **Direction:** They tend to be consistently in one direction, either positive or negative.\n*   **Sources:** They can arise from imperfect calibration of an instrument, imperfections in the experimental technique, or personal errors